# Use ColumnTransformer instead of OneHotEncoder

### Based on features from Omnidian database 101, we compare K-Nearest Neighbors, Gradient Boosting, Random Forest, Bagging, and Logistic Regression.

In [23]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn import tree
import numpy as np
import pandas as pd
from datetime import datetime
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-paper')

In [2]:
import pickle

Get Data

In [18]:
df101_b = pd.read_csv('data/eda101.csv', parse_dates=['Date_Ticket_Initially_Assigned', 'Date_Ticket_Closed', 'installation_date'])
df101_b.head()

,Ticket_Id,Asset_Id,Root_Cause,Ticket_Creation_Reason,asset_type,latitude,longitude,tilt,azimuth,Ticket_Origin,Date_Ticket_Initially_Assigned,Date_Ticket_Closed,installed_by,installation_date
0,23947,101112604,root_cause_normal_wear_and_tear,communication offline,Residential Solar PV,38.944642,-121.248833,37.0,190.0,origin_omnidian_customer,2018-10-22 17:25:27,2018-11-29,Williams Lifetime Builders Inc. DBA Lifetime S...,2013-04-04
1,23947,101112604,root_cause_normal_wear_and_tear,communication offline,Residential Solar PV,38.944642,-121.248833,37.0,190.0,origin_omnidian_customer,2018-10-22 17:25:27,2018-11-29,Williams Lifetime Builders Inc. DBA Lifetime S...,2013-04-04
2,27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,180.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08
3,27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,180.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08
4,27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,90.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08


In [19]:
df101_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 14 columns):
Ticket_Id                         825 non-null int64
Asset_Id                          825 non-null int64
Root_Cause                        825 non-null object
Ticket_Creation_Reason            825 non-null object
asset_type                        825 non-null object
latitude                          825 non-null float64
longitude                         825 non-null float64
tilt                              825 non-null float64
azimuth                           825 non-null float64
Ticket_Origin                     825 non-null object
Date_Ticket_Initially_Assigned    825 non-null datetime64[ns]
Date_Ticket_Closed                825 non-null datetime64[ns]
installed_by                      825 non-null object
installation_date                 825 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(4), int64(2), object(5)
memory usage: 90.3+ KB


Let's change everything to lowercase to reduce confusions.

In [20]:
df101_b.rename(str.lower, axis='columns', inplace=True)

Both ticket_id and asset_id need to be strings

In [86]:
df101_b[['ticket_id', 'asset_id']] = df101_b[['ticket_id', 'asset_id']].astype(str)
df101_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 14 columns):
ticket_id                         825 non-null object
asset_id                          825 non-null object
root_cause                        825 non-null object
ticket_creation_reason            825 non-null object
asset_type                        825 non-null object
latitude                          825 non-null float64
longitude                         825 non-null float64
tilt                              825 non-null float64
azimuth                           825 non-null float64
ticket_origin                     825 non-null object
date_ticket_initially_assigned    825 non-null datetime64[ns]
date_ticket_closed                825 non-null datetime64[ns]
installed_by                      825 non-null object
installation_date                 825 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(4), object(7)
memory usage: 90.3+ KB


In [87]:
df101_b.isnull().any()

ticket_id                         False
asset_id                          False
root_cause                        False
ticket_creation_reason            False
asset_type                        False
latitude                          False
longitude                         False
tilt                              False
azimuth                           False
ticket_origin                     False
date_ticket_initially_assigned    False
date_ticket_closed                False
installed_by                      False
installation_date                 False
dtype: bool

In [81]:
df101_b.iloc[689]

ticket_id                                                  19046
asset_id                                               101112502
root_cause                        root_cause_non-service_support
ticket_creation_reason              inquiry about service ticket
asset_type                                  Residential Solar PV
latitude                                                 39.9235
longitude                                               -74.7514
tilt                                                          30
azimuth                                                      160
ticket_origin                           origin_omnidian_customer
date_ticket_initially_assigned               2018-08-10 17:54:04
date_ticket_closed                           2018-08-10 00:00:00
installed_by                               Solar Energy World NJ
installation_date                            2012-12-03 00:00:00
Name: 689, dtype: object

Assign target to Root_Cause and Train-Test-Split. We'll also take the ticket_id off now so we can use it later to look rows up.

In [62]:
X = df101_b.drop(['root_cause'], axis=1).copy()
y = df101_b['root_cause']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

Let's try ColumnTransformer. We'll pull ticket_id from the dataframes to keep them from being encoded, then we'll put them back together for our function later.

In [65]:
train_ticket = X_train.ticket_id
X_train.drop(['ticket_id'], axis=1)
train_ticket

239    20902
689    19046
644     8381
342     8324
299    13319
383    11743
79      1973
148    18256
363    11041
227    26465
675    11475
332    16651
198     2743
761    10912
464    13103
133    29474
395    29324
346     7909
500    11865
55     18619
633    19228
623     7494
530     5083
570    27825
388     6065
784     7622
306    21488
477    10046
60     13056
714     7511
       ...  
313    31985
459    12775
160     5101
276    21354
191    12475
385    15175
413    22157
491    10088
343     8324
769    20888
308    13722
661    15466
130    18475
663     8638
99     10551
372    29240
87      9456
458    12775
330    12604
214    11873
466    26833
121    16138
614     2926
20     10657
700    18107
71      6904
106    10663
270     3853
435    10913
102     8137
Name: ticket_id, Length: 660, dtype: object

In [66]:
test_ticket = X_test.ticket_id
X_test.drop(['ticket_id'], axis=1)
test_ticket

611    15895
174     2591
67     11278
168    16584
275    11859
336    18909
811    17847
86     29226
541     7791
716    16078
436    10913
265    29244
412    22157
30     16499
380    20595
669    28625
39     10747
213    11667
787     7622
362    25770
786     7622
285    27929
709    29254
552    15458
420    10417
737    12803
432    29236
254    21911
640    14246
155    29251
       ...  
411    22303
81     22782
775    24158
823    32514
722    25704
605    22310
706    13997
326    13549
808    13116
423    23113
447    18254
360    11253
759     6360
396    25452
7       8568
316    17709
101    12571
806    13135
314    30721
792     9282
584     7972
2      27384
712     8479
760     7561
581    30112
290    10361
211    18114
481    24754
768    22311
527    29239
Name: ticket_id, Length: 165, dtype: object

In [33]:
# List our categorical features
categoricals = list(X_test.columns[(X_test.dtypes.values == np.dtype('object'))])
categoricals

['asset_id',
 'ticket_creation_reason',
 'asset_type',
 'ticket_origin',
 'installed_by']

In [56]:
X_nums = list(X_test.columns[(X_test.dtypes.values != np.dtype('object'))])
X_nums

['latitude',
 'longitude',
 'tilt',
 'azimuth',
 'date_ticket_initially_assigned',
 'date_ticket_closed',
 'installation_date']

In [59]:
X_train_num = X_train[X_nums].copy()
X_train_num.head()

,latitude,longitude,tilt,azimuth,date_ticket_initially_assigned,date_ticket_closed,installation_date
239,34.038597,-118.493806,40.0,225.0,2018-08-31 21:08:05,2018-11-20,2014-03-26
689,39.923479,-74.751410,30.0,160.0,2018-08-10 17:54:04,2018-08-10,2012-12-03
644,38.251630,-122.149367,23.0,152.0,2018-01-22 18:46:21,2018-03-08,2014-10-07
342,34.448895,-119.260475,9.0,180.0,2018-01-24 00:55:37,2018-03-09,2014-02-18
299,34.138321,-117.556590,23.0,180.0,2018-04-24 23:00:48,2018-06-23,2015-02-11


In [46]:
preprocessor = make_column_transformer( (OneHotEncoder(handle_unknown='ignore'), categoricals))
encoder = preprocessor.fit(X_train)

In [47]:
X_train_enc = pd.DataFrame(encoder.transform(X_train).toarray(),
                         columns=encoder.get_feature_names())
X_test_enc = pd.DataFrame(encoder.transform(X_test).toarray(),
                        columns=encoder.get_feature_names())

In [71]:
X_train_enc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Columns: 379 entries, onehotencoder__x0_101111473 to onehotencoder__x4_Williams Lifetime Builders Inc. DBA Lifetime Solar
dtypes: float64(379)
memory usage: 1.9 MB


Now we'll put the ticket_id, numerical columns, and encoded columns all together.

In [69]:
X_train_num.insert(loc=0, column='ticket_id', value=train_ticket)
X_train_num.head()

,ticket_id,latitude,longitude,tilt,azimuth,date_ticket_initially_assigned,date_ticket_closed,installation_date
239,20902,34.038597,-118.493806,40.0,225.0,2018-08-31 21:08:05,2018-11-20,2014-03-26
689,19046,39.923479,-74.751410,30.0,160.0,2018-08-10 17:54:04,2018-08-10,2012-12-03
644,8381,38.251630,-122.149367,23.0,152.0,2018-01-22 18:46:21,2018-03-08,2014-10-07
342,8324,34.448895,-119.260475,9.0,180.0,2018-01-24 00:55:37,2018-03-09,2014-02-18
299,13319,34.138321,-117.556590,23.0,180.0,2018-04-24 23:00:48,2018-06-23,2015-02-11


In [70]:
X_train_mega = X_train_num.join(X_train_enc)
X_train_mega.head()

,ticket_id,latitude,longitude,tilt,azimuth,date_ticket_initially_assigned,date_ticket_closed,installation_date,onehotencoder__x0_101111473,onehotencoder__x0_101111478,...,onehotencoder__x4_Solar Alliance of America Inc.,onehotencoder__x4_Solar Energy World NJ,onehotencoder__x4_Solar Plus LLC,onehotencoder__x4_Sonic Solar Energy,onehotencoder__x4_Summerwindsolar LLC Phoenix,onehotencoder__x4_Summit Technology Group,onehotencoder__x4_SunStarter Solar Installations Inc,onehotencoder__x4_Syntrol Plumbing Heating and Air,onehotencoder__x4_Talbott Solar Home,onehotencoder__x4_Williams Lifetime Builders Inc. DBA Lifetime Solar
239,20902,34.038597,-118.493806,40.0,225.0,2018-08-31 21:08:05,2018-11-20,2014-03-26,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
689,19046,39.923479,-74.751410,30.0,160.0,2018-08-10 17:54:04,2018-08-10,2012-12-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,8381,38.251630,-122.149367,23.0,152.0,2018-01-22 18:46:21,2018-03-08,2014-10-07,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
342,8324,34.448895,-119.260475,9.0,180.0,2018-01-24 00:55:37,2018-03-09,2014-02-18,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299,13319,34.138321,-117.556590,23.0,180.0,2018-04-24 23:00:48,2018-06-23,2015-02-11,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
X_train_mega.isna().any().sum()

379

In [90]:
X_train_mega[X_train_mega.ticket_id == 19046]

,ticket_id,latitude,longitude,tilt,azimuth,date_ticket_initially_assigned,date_ticket_closed,installation_date,onehotencoder__x0_101111473,onehotencoder__x0_101111478,...,onehotencoder__x4_Solar Alliance of America Inc.,onehotencoder__x4_Solar Energy World NJ,onehotencoder__x4_Solar Plus LLC,onehotencoder__x4_Sonic Solar Energy,onehotencoder__x4_Summerwindsolar LLC Phoenix,onehotencoder__x4_Summit Technology Group,onehotencoder__x4_SunStarter Solar Installations Inc,onehotencoder__x4_Syntrol Plumbing Heating and Air,onehotencoder__x4_Talbott Solar Home,onehotencoder__x4_Williams Lifetime Builders Inc. DBA Lifetime Solar
689,19046,39.923479,-74.75141,30.0,160.0,2018-08-10 17:54:04,2018-08-10,2012-12-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690,19046,39.923479,-74.75141,30.0,160.0,2018-08-10 17:54:04,2018-08-10,2012-12-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
X_train_mega.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 239 to 102
Columns: 387 entries, ticket_id to onehotencoder__x4_Williams Lifetime Builders Inc. DBA Lifetime Solar
dtypes: datetime64[ns](3), float64(383), object(1)
memory usage: 2.0+ MB


### Modeling

In [50]:
gb = GradientBoostingClassifier(random_state=42, min_samples_leaf=30)
gb.fit(X_train_enc, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=30, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [51]:
gb.score(X_test_enc, y_test)

0.5878787878787879

One hot encoding: since our data is largely categorical but our models require numeric inputs, we one hot encode it.


In [16]:
# encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
# encoder.fit(X_train)

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [17]:
# X_train = pd.DataFrame(encoder.transform(X_train).toarray(),
#                          columns=encoder.get_feature_names())
# X_test = pd.DataFrame(encoder.transform(X_test).toarray(),
#                         columns=encoder.get_feature_names())

Training our models.

In [18]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [19]:
# filename = '101_knn.pkl'
# pickle.dump(knn, open(filename, 'wb'))

In [20]:
# kn = pickle.load

In [21]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
# filename = '101_lr.pkl'
# pickle.dump(lr, open(filename, 'wb'))

In [23]:
gb = GradientBoostingClassifier(random_state=42, min_samples_leaf=30)
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=30, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [24]:
# filename = '101_gb.pkl'
# pickle.dump(gb, open(filename, 'wb'))

In [25]:
dt = DecisionTreeClassifier(random_state=42, min_samples_leaf=30)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=30, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [26]:
# filename = '101_dt.pkl'
# pickle.dump(dt, open(filename, 'wb'))

In [27]:
bg = BaggingClassifier(random_state=25565)
bg.fit(X_train, y_train)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=25565, verbose=0,
                  warm_start=False)

In [28]:
# filename = '101_bg.pkl'
# pickle.dump(bg, open(filename, 'wb'))

In [29]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)
rf.fit(X_train, y_train);

In [30]:
# filename = '101_rf.pkl'
# pickle.dump(rf, open(filename, 'wb'))

Let's show our results

In [31]:
knn.score(X_test, y_test)

0.5818181818181818

In [32]:
lr.score(X_test, y_test)

0.8242424242424242

In [33]:
gb.score(X_test, y_test)

0.6121212121212121

In [34]:
dt.score(X_test, y_test)

0.593939393939394

In [35]:
bg.score(X_test, y_test)

0.8363636363636363

In [36]:
rf.score(X_test, y_test)

0.8303030303030303

What may be some other classifiers?

In [48]:
from sklearn.utils.testing import all_estimators

In [56]:
rf.predict_proba(X_test)[0]

array([0.  , 0.  , 0.  , 0.01, 0.  , 0.11, 0.  , 0.88, 0.  , 0.  ])

In [60]:
rf.predict(X_test)[0:5]

array(['root_cause_normal_wear_and_tear',
       'root_cause_major_component_failure_warranty',
       'root_cause_normal_wear_and_tear',
       'root_cause_normal_wear_and_tear',
       'root_cause_major_component_failure_warranty'], dtype=object)

In [94]:
l_props = lr.predict_proba(X_test)[1]

In [100]:
zipp = list(zip(lr.classes_, l_props))

In [11]:
zipp

NameError: name 'zipp' is not defined

In [110]:
#want a dictionary that gives probability for each class, 
#prediction, and ground truth
def display_preds_truth(model, obs, X_test, y_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
#     display['prediction'] = model.predict(X_test)[obs]
    display['ground truth'] = y_test[obs]
    return display
        
    
    

In [129]:
#want a dictionary that gives probability for each class,
def display_probas(model, obs, X_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
    display['prediction'] = model.predict(X_test)[obs]
    return display

In [130]:
display_probas(lr, 1, X_test)

{'root_cause_design/sale_issue': 0.00043387759236169344,
 'root_cause_environmental': 0.004206441640046431,
 'root_cause_homeowner': 0.014217294259570568,
 'root_cause_installer_workmanship': 0.004648355311296379,
 'root_cause_major_component_failure_non-warranty': 0.0036398455245150647,
 'root_cause_major_component_failure_warranty': 0.9000584314884308,
 'root_cause_non-service_support': 0.027058029061024055,
 'root_cause_normal_wear_and_tear': 0.0411124777776664,
 'root_cause_roof_issue': 0.0027033523841786256,
 'root_cause_service_workmanship': 0.0019218949609099132,
 'prediction': 'root_cause_major_component_failure_warranty'}

In [131]:
X_test

,x0_1866,x0_2015,x0_2591,x0_2743,x0_2766,x0_2809,x0_2820,x0_2926,x0_3220,x0_3664,...,x18_2016-03-09 00:00:00,x18_2016-03-16 00:00:00,x18_2016-03-21 00:00:00,x18_2016-03-22 00:00:00,x18_2016-04-27 00:00:00,x18_2016-05-11 00:00:00,x18_2016-05-18 00:00:00,x18_2016-06-29 00:00:00,x18_2016-07-01 00:00:00,x18_2016-08-03 00:00:00
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
y_test

714                    root_cause_normal_wear_and_tear
605        root_cause_major_component_failure_warranty
120                    root_cause_normal_wear_and_tear
208                    root_cause_normal_wear_and_tear
380        root_cause_major_component_failure_warranty
816        root_cause_major_component_failure_warranty
575                    root_cause_normal_wear_and_tear
266        root_cause_major_component_failure_warranty
557                    root_cause_normal_wear_and_tear
825                    root_cause_normal_wear_and_tear
580                    root_cause_normal_wear_and_tear
327                    root_cause_normal_wear_and_tear
623    root_cause_major_component_failure_non-warranty
365        root_cause_major_component_failure_warranty
827                     root_cause_non-service_support
198                    root_cause_normal_wear_and_tear
456                    root_cause_normal_wear_and_tear
465                    root_cause_normal_wear_and_tear
311       